## Salvar comunicados e atas do Copom em português e ingles

In [2]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
from pandas.io.json import json_normalize
import json
import itertools
from datetime import datetime

In [3]:
# Cria uma pasta local para salvar os arquivos
if not os.path.exists('comunicados/comun_pt'):
    os.makedirs('comunicados/comun_pt')
if not os.path.exists('comunicados/comun_en'):
    os.makedirs('comunicados/comun_en')
if not os.path.exists('atas/atas_pt'):
    os.makedirs('atas/atas_pt')
if not os.path.exists('atas/atas_en'):
    os.makedirs('atas/atas_en')

In [4]:
# URL da página
url_ata_pt_api = 'https://www.bcb.gov.br/api/servico/sitebcb/atascopom/ultimas'
url_ata_en_api = 'https://www.bcb.gov.br/api/servico/sitebcb/copomminutes/ultimas'
url_comun_en_api = 'https://www.bcb.gov.br/api/servico/sitebcb/copomstatements/'
url_comun_pt_api = 'https://www.bcb.gov.br/api/servico/sitebcb/comunicadoscopom/'

In [5]:
this_year = datetime.now().year
period = this_year - 1999
years = list(range(this_year, this_year - period, -1))

In [6]:
def scrape_ata_pt():
    cookies = {
    '_ga': 'GA1.3.883152977.1659640860',
    '_gid': 'GA1.3.167621744.1678443576',
    'bcb-aceitacookie': 'true',
    '_gat_gtag_UA_65460906_3': '1',
    }

    headers = {
        'authority': 'www.bcb.gov.br',
        'accept': 'application/json, text/plain, */*',
        'accept-language': 'en-US,en;q=0.9,es;q=0.8,fr;q=0.7,pt;q=0.6,it;q=0.5,la;q=0.4',
        # 'cookie': '_ga=GA1.3.883152977.1659640860; _gid=GA1.3.167621744.1678443576; bcb-aceitacookie=true; _gat_gtag_UA_65460906_3=1',
        'if-none-match': '"4f9bf307-6c2b-40df-a813-e2cc883630b4"',
        'referer': 'https://www.bcb.gov.br/publicacoes/atascopom/cronologicos',
        'sec-ch-ua': '"Chromium";v="110", "Not A(Brand";v="24", "Google Chrome";v="110"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-origin',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36',
    }

    params = {
        'quantidade': '1000',
        'filtro': '',
    }

    response = requests.get(
        url_ata_pt_api,
        params=params,
        cookies=cookies,
        headers=headers,
    )
    # Cria um objeto BeautifulSoup para fazer o parse do HTML
    soup = BeautifulSoup(response.content, 'html.parser')
    a = "[" + soup.text[13:-2] + "]"
    try:
        dict_obj = json.loads(a)
    except json.JSONDecodeError:
        dict_obj = {}
    df = pd.DataFrame(dict_obj)
    
    return df

In [7]:
def scrape_ata_en():
    cookies = {
    'bcb-aceitacookie': 'true',
    '_ga_JE5VH519BS': 'GS1.1.1684500328.1.0.1684500556.0.0.0',
    '_gid': 'GA1.3.1153499871.1686079398',
    '_ga_TPS4R7DC1S': 'GS1.1.1686079397.14.1.1686079520.0.0.0',
    '_ga': 'GA1.3.883152977.1659640860',
    '_gat_gtag_UA_65460906_3': '1',
    }

    headers = {
        'authority': 'www.bcb.gov.br',
        'accept': 'application/json, text/plain, */*',
        'accept-language': 'en-US,en;q=0.9,es;q=0.8,fr;q=0.7,pt;q=0.6,it;q=0.5,la;q=0.4',
        # 'cookie': 'bcb-aceitacookie=true; _ga_JE5VH519BS=GS1.1.1684500328.1.0.1684500556.0.0.0; _gid=GA1.3.1153499871.1686079398; _ga_TPS4R7DC1S=GS1.1.1686079397.14.1.1686079520.0.0.0; _ga=GA1.3.883152977.1659640860; _gat_gtag_UA_65460906_3=1',
        'if-none-match': '"03675e2d-0783-44f5-817a-9f1bd9c686ba"',
        'referer': 'https://www.bcb.gov.br/en/publications/copomminutes/cronologicos',
        'sec-ch-ua': '"Not.A/Brand";v="8", "Chromium";v="114", "Google Chrome";v="114"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-origin',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
    }

    params = {
        'quantidade': '1000',
        'filtro': '',
    }

    response = requests.get(
        url_ata_en_api,
        params=params,
        cookies=cookies,
        headers=headers,
    )
    
    # Cria um objeto BeautifulSoup para fazer o parse do HTML
    soup = BeautifulSoup(response.content, 'html.parser')
    a = "[" + soup.text[13:-2] + "]"
    try:
        dict_obj = json.loads(a)
    except json.JSONDecodeError:
        dict_obj = {}
    df = pd.DataFrame(dict_obj)
    
    return df

In [8]:
def scrape_comun_pt(year):
    cookies = {
        'bcb-aceitacookie': 'true',
        '_gid': 'GA1.3.1153499871.1686079398',
        '_ga_JE5VH519BS': 'GS1.1.1686088498.3.1.1686088526.0.0.0',
        '_ga_K36K6BCHRD': 'GS1.1.1686088604.1.1.1686088662.0.0.0',
        '_gat_UA-65460906-3': '1',
        '_ga': 'GA1.1.883152977.1659640860',
        '_ga_TPS4R7DC1S': 'GS1.1.1686088498.15.1.1686088800.0.0.0',
    }

    headers = {
        'authority': 'www.bcb.gov.br',
        'accept': 'application/json, text/plain, */*',
        'accept-language': 'en-US,en;q=0.9,es;q=0.8,fr;q=0.7,pt;q=0.6,it;q=0.5,la;q=0.4',
        # 'cookie': 'bcb-aceitacookie=true; _gid=GA1.3.1153499871.1686079398; _ga_JE5VH519BS=GS1.1.1686088498.3.1.1686088526.0.0.0; _ga_K36K6BCHRD=GS1.1.1686088604.1.1.1686088662.0.0.0; _gat_UA-65460906-3=1; _ga=GA1.1.883152977.1659640860; _ga_TPS4R7DC1S=GS1.1.1686088498.15.1.1686088800.0.0.0',
        'if-none-match': '"e47eab8f-a1f1-43f1-afbf-8a2548b8d4c2"',
        'referer': 'https://www.bcb.gov.br/controleinflacao/comunicadoscopom',
        'sec-ch-ua': '"Not.A/Brand";v="8", "Chromium";v="114", "Google Chrome";v="114"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-origin',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
    }

    params = {
        'ano': str(year),
    }

    response = requests.get(
        url_comun_pt_api,
        params=params,
        cookies=cookies,
        headers=headers,
    )
    
    # Cria um objeto BeautifulSoup para fazer o parse do HTML
    soup = BeautifulSoup(response.content, 'html.parser')
    a = "[" + soup.text[13:-2] + "]"
    try:
        dict_obj = json.loads(a)
    except json.JSONDecodeError:
        dict_obj = {}
    df = pd.DataFrame(dict_obj)
    
    return df

In [9]:
def scrape_comun_en(year):
    cookies = {
        'bcb-aceitacookie': 'true',
        '_gid': 'GA1.3.1153499871.1686079398',
        '_ga_JE5VH519BS': 'GS1.1.1686088498.3.1.1686088526.0.0.0',
        '_ga_K36K6BCHRD': 'GS1.1.1686088604.1.1.1686088662.0.0.0',
        '_gat': '1',
        '_ga': 'GA1.1.883152977.1659640860',
        '_ga_TPS4R7DC1S': 'GS1.1.1686244204.20.1.1686244387.0.0.0',
    }

    headers = {
        'authority': 'www.bcb.gov.br',
        'accept': 'application/json, text/plain, */*',
        'accept-language': 'en-US,en;q=0.9,es;q=0.8,fr;q=0.7,pt;q=0.6,it;q=0.5,la;q=0.4',
        # 'cookie': 'bcb-aceitacookie=true; _gid=GA1.3.1153499871.1686079398; _ga_JE5VH519BS=GS1.1.1686088498.3.1.1686088526.0.0.0; _ga_K36K6BCHRD=GS1.1.1686088604.1.1.1686088662.0.0.0; _gat=1; _ga=GA1.1.883152977.1659640860; _ga_TPS4R7DC1S=GS1.1.1686244204.20.1.1686244387.0.0.0',
        'if-none-match': '"de84e3a3-2b0d-49c5-a9ac-44a32109859e"',
        'referer': 'https://www.bcb.gov.br/en/monetarypolicy/copomstatements',
        'sec-ch-ua': '"Not.A/Brand";v="8", "Chromium";v="114", "Google Chrome";v="114"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-origin',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
    }

    params = {
        'ano': year,
    }

    response = requests.get(
        'https://www.bcb.gov.br/api/servico/sitebcb/copomstatements',
        params=params,
        cookies=cookies,
        headers=headers,
    )
    
    # Cria um objeto BeautifulSoup para fazer o parse do HTML
    soup = BeautifulSoup(response.content, 'html.parser')
    a = "[" + soup.text[13:-2] + "]"
    try:
        dict_obj = json.loads(a)
    except json.JSONDecodeError:
        dict_obj = {}
    df = pd.DataFrame(dict_obj)
    
    return df

In [ ]:
## scrape atas do Copom em portugues

df = scrape_ata_pt()
df['LinkPagina'] = 'https://www.bcb.gov.br' + df['LinkPagina']
urls = df['LinkPagina'].tolist()

for url in urls:
    id = url[-8:]
    url_temp = 'https://www.bcb.gov.br/api/servico/sitebcb/atascopom/principal?filtro=IdentificadorUrl%20eq%20%27' + id + '%27'
    response = requests.get(url_temp)
    soup = BeautifulSoup(response.content, 'html.parser')
    ata = soup.text[12:-1]
    
    try:
        dict_obj = json.loads(ata)
    except json.JSONDecodeError:
        dict_obj = {}
    filename = id
    with open(f'atas/atas_pt/{filename}_pt.json', 'w') as f:
        json.dump(dict_obj, f)

In [ ]:
## scrape atas do Copom em ingles

df = scrape_ata_en()
df['LinkPagina'] = 'https://www.bcb.gov.br' + df['LinkPagina']
urls = df['LinkPagina'].tolist()

for url in urls:
    id = url[-8:]
    url_temp = 'https://www.bcb.gov.br/api/servico/sitebcb/copomminutes/principal?filtro=IdentificadorUrl%20eq%20%27' + id + '%27'
    response = requests.get(url_temp)
    soup = BeautifulSoup(response.content, 'html.parser')
    ata = soup.text[12:-1]

    try:
        dict_obj = json.loads(ata)
    except json.JSONDecodeError:
        dict_obj = {}
    filename = id
    with open(f'atas/atas_en/{filename}_en.json', 'w') as f:
        json.dump(dict_obj, f)

In [ ]:
## scrape comunicados do Copom em portugues

for year in years:
    
    df = scrape_comun_pt(year)
    
    try:
        urls = df['Url'].tolist()
        filenames = df['notaCopom'].astype(int).tolist()
    except:
        break
    
    for url, filename in zip(urls, filenames):
        id = url[-10:-5]
        url_temp = url_comun_pt_api + id
        response = requests.get(url_temp)
        soup = BeautifulSoup(response.content, 'html.parser')
        comunicado = soup.text[12:-1]
        try:
            dict_obj = json.loads(comunicado)
        except json.JSONDecodeError:
            dict_obj = {}
        filename = filename
        with open(f'comunicados/comun_pt/{filename}_pt.json', 'w') as f:
            json.dump(dict_obj, f)

In [56]:
df_comun_pt = pd.DataFrame()

for year in years:
    
    df = scrape_comun_pt(year)
        
    try:
        urls = df['Url'].tolist()
        filenames = df['notaCopom'].astype(int).tolist()
    except:
        break
    
    for url, filename in zip(urls, filenames):
        idd = url[-10:-5]
        url_temp = url_comun_pt_api + idd
        try:
            df_2 = pd.read_json(url_temp)
            df_3 = json_normalize(df_2["conteudo"])
        except:
            pass
        df_comun_pt = pd.concat([df_comun_pt,df_3])

17886
17857
17825
17794
17753
17731
17704
17683
17663
17637
17614
17585
17545
17510
17461
17422
17388
17341
17297
17268
17224
17188
17148
17095
17067
17003
16964
16924
16888
16857
16812
16780
16745
16703
16675
16633
16595
16569
16538
16512
16475
16417
16381
16340
16298
16260
16217
16180
16130
16088
16054
16023
15966
15961
15960
15959
15958
15957
15956
15955
15954
15953
15952
15951
15950
15949
15948
15947
15946
15945
15944
15943
15942
15941
15940
15939
15938
15937
15936
15935
15934
15933
15932
15931
15930
15929
15928
15927
15926
15925
15924
15923
15922
15921
15920
15919
15918
15917
15916
15915
15914
15913
15912
15911
15910
15909
15908
15907
15906
15905
15904
15903
15902
15901
15900
15899
15898
15897
15896
15895
15894
15893
15892
15891
15890
15889
15888
15887
15886
15885
15884
15883
15882
15881
15880
15879
15878
15877
15876
15875
15874
15873
15872
15871
15870
15869
15868
15867
15866
15865
15864
15863
15862
15861
15860
15859
15858
15857
15856
15855
15854
15853
15852
15851
15850
15849
1584

,Conteudo,Titulo,dataPublicacao,mostrarCabecalho,mostrarRodape,ultimaAtualizacao
0,"<p style=""text-align&#58;justify;"">O ambiente ...","Copom mantém a taxa Selic em 13,75% a.a.",2023-05-03T21:46:13Z,False,False,2023-05-03T21:46:14Z
0,"<p style=""text-align&#58;justify;"">Desde a reu...","Copom mantém a taxa Selic em 13,75% a.a.",2023-03-22T21:31:40Z,False,False,2023-03-22T21:31:41Z
0,"<p style=""text-align&#58;justify;"">Em sua 252ª...","Copom mantém a taxa Selic em 13,75% a.a.",2023-02-01T21:34:30Z,False,False,2023-02-01T21:34:30Z
0,"<p style=""text-align&#58;justify;"">Em sua 251ª...","Copom mantém a taxa Selic em 13,75% a.a.",2022-12-07T21:31:04Z,False,False,2022-12-07T21:31:04Z
0,"<p style=""text-align&#58;justify;"">Em sua 250ª...","Copom mantém a taxa Selic em 13,75% a.a.",2022-10-26T21:35:29Z,False,False,2022-10-26T21:35:30Z
0,"<p style=""text-align&#58;justify;"">Em sua 249ª...","Copom mantém a taxa Selic em 13,75% a.a.",2022-09-21T21:34:28Z,False,False,2022-09-21T21:34:29Z
0,"<p style=""text-align&#58;justify;"">Em sua 248ª...","Copom eleva a taxa Selic para 13,75% a.a.",2022-08-03T21:31:00Z,False,False,2022-08-03T21:48:25Z
0,"<p>​</p><p style=""text-align&#58;justify;"">Em ...","Copom eleva a taxa Selic para 13,25% a.a.",2022-06-15T21:36:14Z,False,False,2022-06-15T21:36:14Z
0,"<p>​</p><p style=""text-align&#58;justify;"">Em ...","Copom eleva a taxa Selic para 12,75% a.a.",2022-05-04T22:12:11Z,False,False,2022-05-04T22:12:11Z
0,"<p style=""text-align&#58;justify;"">Em sua 245ª...","Copom eleva a taxa Selic para 11,75% a.a.",2022-03-16T22:13:12Z,False,False,2022-03-16T22:13:13Z


In [58]:
df_comun_pt.to_csv('comunicados.csv', index=False)

In [ ]:
## scrape comunicados do Copom em inglês

df_comun_en = pd.DataFrame()

for year in years:
    
    df = scrape_comun_en(year)
        
    try:
        urls = df['Url'].tolist()
        filenames = df['notaCopom'].astype(int).tolist()
    except:
        break
    
    for url, filename in zip(urls, filenames):
        idd = url[-10:-5]
        url_temp = url_comun_en_api + idd
        df_2 = pd.read_json(url_temp)
        df_3 = json_normalize(df_2["conteudo"])
        df_comun_en = pd.concat([df_comun_en,df_3])

In [57]:
df_comun_en.to_csv('statements.csv', index=False)

NameError: name 'df_comun_en' is not defined

In [10]:
df_comun_pt = pd.DataFrame()
df_comun_pt

""


In [38]:
df = scrape_comun_pt("2009")
df

,DataModificacao,DataPublicacao,Id,Url,dataCriacao,dataReferencia,descricao,notaCopom,titulo
0,2016-10-18T20:01:24Z,2009-12-09T02:00:00Z,15907,/detalhenoticia/15907/nota,2016-10-18T20:01:24Z,2009-12-09T02:00:00Z,,147.0,147ª reunião - Nota à Imprensa - 147ª. Reunião
1,2016-10-18T20:01:24Z,2009-10-21T02:00:00Z,15906,/detalhenoticia/15906/nota,2016-10-18T20:01:24Z,2009-10-21T02:00:00Z,,146.0,146ª reunião - Nota à Imprensa - 146ª. Reunião
2,2016-10-18T20:01:24Z,2009-09-02T03:00:00Z,15905,/detalhenoticia/15905/nota,2016-10-18T20:01:24Z,2009-09-02T03:00:00Z,,145.0,145ª reunião - Nota à Imprensa - 145ª. Reunião
3,2016-10-18T20:01:24Z,2009-07-22T03:00:00Z,15904,/detalhenoticia/15904/nota,2016-10-18T20:01:24Z,2009-07-22T03:00:00Z,,144.0,144ª reunião - Nota à Imprensa - 144ª. Reunião
4,2016-10-18T20:01:24Z,2009-06-10T03:00:00Z,15903,/detalhenoticia/15903/nota,2016-10-18T20:01:24Z,2009-06-10T03:00:00Z,,143.0,143ª reunião - Nota à Imprensa - 143ª. Reunião
5,2016-10-18T20:01:24Z,2009-04-29T03:00:00Z,15902,/detalhenoticia/15902/nota,2016-10-18T20:01:24Z,2009-04-29T03:00:00Z,,142.0,142ª reunião - Nota à Imprensa - 142ª. Reunião
6,2016-10-18T20:01:24Z,2009-03-11T03:00:00Z,15901,/detalhenoticia/15901/nota,2016-10-18T20:01:24Z,2009-03-11T03:00:00Z,,141.0,141ª reunião - Nota à Imprensa - 141ª. Reunião
7,2016-10-18T20:01:24Z,2009-01-21T02:00:00Z,15900,/detalhenoticia/15900/nota,2016-10-18T20:01:24Z,2009-01-21T02:00:00Z,,140.0,140ª reunião - Nota à Imprensa - 140ª. Reunião


In [39]:
urls = df['Url'].tolist()
filenames = df['notaCopom'].astype(int).tolist()
urls, filenames

(['/detalhenoticia/15907/nota',
  '/detalhenoticia/15906/nota',
  '/detalhenoticia/15905/nota',
  '/detalhenoticia/15904/nota',
  '/detalhenoticia/15903/nota',
  '/detalhenoticia/15902/nota',
  '/detalhenoticia/15901/nota',
  '/detalhenoticia/15900/nota'],
 [147, 146, 145, 144, 143, 142, 141, 140])

In [51]:
idd = urls[4][-10:-5]
idd 

'15903'

In [52]:
url_temp = url_comun_pt_api + idd
url_temp

'https://www.bcb.gov.br/api/servico/sitebcb/comunicadoscopom/15903'

In [53]:
df_2 = pd.read_json(url_temp)
df_2

,conteudo
0,"{'Titulo': 'Nota à Imprensa - 143ª. Reunião', ..."


In [54]:
df_3 = json_normalize(df_2["conteudo"])
df_3

,Conteudo,Titulo,dataPublicacao,mostrarCabecalho,mostrarRodape,ultimaAtualizacao
0,<div> <p>Tendo em vista as perspectivas para a...,Nota à Imprensa - 143ª. Reunião,2009-06-10T03:00:00Z,False,False,2016-10-18T20:01:24Z


In [55]:
df_comun_pt = pd.concat([df_comun_pt,df_3])

df_comun_pt

,Conteudo,Titulo,dataPublicacao,mostrarCabecalho,mostrarRodape,ultimaAtualizacao
0,"<p style=""text-align&#58;justify;"">O ambiente ...","Copom mantém a taxa Selic em 13,75% a.a.",2023-05-03T21:46:13Z,False,False,2023-05-03T21:46:14Z
0,"<p style=""text-align&#58;justify;"">Desde a reu...","Copom mantém a taxa Selic em 13,75% a.a.",2023-03-22T21:31:40Z,False,False,2023-03-22T21:31:41Z
0,"<p style=""text-align&#58;justify;"">Em sua 252ª...","Copom mantém a taxa Selic em 13,75% a.a.",2023-02-01T21:34:30Z,False,False,2023-02-01T21:34:30Z
0,"<p style=""text-align&#58;justify;"">Em sua 251ª...","Copom mantém a taxa Selic em 13,75% a.a.",2022-12-07T21:31:04Z,False,False,2022-12-07T21:31:04Z
0,"<p style=""text-align&#58;justify;"">Em sua 250ª...","Copom mantém a taxa Selic em 13,75% a.a.",2022-10-26T21:35:29Z,False,False,2022-10-26T21:35:30Z
0,"<p style=""text-align&#58;justify;"">Em sua 249ª...","Copom mantém a taxa Selic em 13,75% a.a.",2022-09-21T21:34:28Z,False,False,2022-09-21T21:34:29Z
0,"<p style=""text-align&#58;justify;"">Em sua 248ª...","Copom eleva a taxa Selic para 13,75% a.a.",2022-08-03T21:31:00Z,False,False,2022-08-03T21:48:25Z
0,"<p>​</p><p style=""text-align&#58;justify;"">Em ...","Copom eleva a taxa Selic para 13,25% a.a.",2022-06-15T21:36:14Z,False,False,2022-06-15T21:36:14Z
0,"<p>​</p><p style=""text-align&#58;justify;"">Em ...","Copom eleva a taxa Selic para 12,75% a.a.",2022-05-04T22:12:11Z,False,False,2022-05-04T22:12:11Z
0,"<p style=""text-align&#58;justify;"">Em sua 245ª...","Copom eleva a taxa Selic para 11,75% a.a.",2022-03-16T22:13:12Z,False,False,2022-03-16T22:13:13Z
